In [2]:
# conda install -c conda-forge python-annoy
# conda install -c conda-forge pymorphy2
# conda install -c conda-forge stop-words
# conda install -c conda-forge gensim
# conda install -c conda-forge ipywidgets

# pip install chardet для определения кодировщика

Учебный проект №14:

классификатор «товарный запрос vs. болталка»

In [1]:
# необходимые модули
import os
import string
import annoy

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

import pandas as pd 
from tqdm import tqdm, notebook
import time

import numpy as np
import pandas as pd
import codecs
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

# у меня все гигинтские файлы лежат за пределами папки, которая сохраняется в GIT
# кто будет пробовать - необходимо уточнить путь к папке с входными файлами
DIR="..\\..\\chunk\\14\\"

сначала идут блоки, необходимые для работы готовой модели,
а так же идет загрузка сохраненных и готовых к работе частей, и сразу можно запускать запросы.
а все подготовительные действия, с помощью которых сохранены части модели, следуют ниже.

вспомогательный обработчик текстов, включающий функции удаления стопслов, знаков пунктуации и приведения слов к начальной форме
для уменьшения размерности корпуса векторов, сопоставляемых набору разных слов

In [2]:
# лемматизация требуется и при подготовке, и при использовании готовой модели
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [4]:
# восстановление обученной логистической регрессии, помогающей отличать продуктовые запросы от всех остальных
clf = pickle.load(open(f'{DIR}LogisticRegression.sav', 'rb'))

In [5]:
# загрузить только минимум для работы.модель word2vec больше не будет дообучатьсяя
modelwv = KeyedVectors.load(f'{DIR}word2vec.wordvectors', mmap='r')

In [6]:
# загрузить индексы
indexProduct = annoy.AnnoyIndex(100 ,'angular')
indexProduct.load(f'{DIR}indexProduct.annoy')
indexFlood = annoy.AnnoyIndex(100 ,'angular')
indexFlood.load(f'{DIR}indexFlood.annoy')

True

In [19]:
# рабочие датасеты для болталки и товаров
# ВАЖНО!индексы датасетов нельзя менять, т.к. annoy сохранен на конкретные индексы в двух датасетах
# при добавлении/удалении рядов из датасетов потребуется переиндексировать и переобучать модель
df=pd.read_csv(f'{DIR}mailruflood.csv',index_col=0)
pds=pd.read_csv(f'{DIR}ProductsDS.csv',index_col=0)

In [8]:
# вспомогательные функции для вычисления на лету вектора запроса и для классификации запроса
def vector_(spls):
    n_w2v = 0
    # вычисление вектора фразы
    vector = np.zeros(100)
    notinvoc=[] #слова не в словаре модели
    for word in spls:
        if word in modelwv:
            vector += modelwv[word]
            n_w2v += 1
        else:
            notinvoc.append(word)
    if n_w2v > 0:
        vector = vector / n_w2v
    # index.add_item(counter, vector)
    # counter += 1
    return vector,notinvoc
def choosetarget(userrequest):
# запрос юзера - строка слов, разделенных пробелами
    line=preprocess_txt(userrequest) #предобработка
    # вычислить вектор фразы для некоего пространства, на котором обучена модель
    # если попадаются слова не из словаря, то выдать диагностическое сообщение
    vector,notinvoc=vector_(line)
    if len(notinvoc)>0:print(f'слова из запроса: <{notinvoc}> отсутствуют в словаре модели, поэтому точность результата снижена!')
    # определить категорию запроса
    y_predict=clf.predict([vector])
    if y_predict==0:
        pass 
        # print('запрос из товарной группы')
        return 0,vector,'запрос из товарной группы'
    else:
        pass 
        # print('запрос не идентифицирован, как товарный, найти ответ из болталки')
        return 1,vector,'запрос не идентифицирован, как товарный, найти ответ из болталки'



вот целевая функция задания:
на вход подается строка-запрос
на выходе самая подходящая по смыслу из имеющихся в базах данных
строка-ответ с учетом классификации запроса на продуктовый/непродуктовый

In [62]:
def get_answer(userrequest,moreproduct=1): #userrequest - слова, разделенные пробелами
    # выдать ответ на запрос
    # сначала классифицировать: продуктовый или нет
    # потом найти в датасетах самый подходящий ответ
    index=[indexProduct,indexFlood]
    ds=[pds,df]
    target,vector,info =choosetarget(userrequest)

    ind0,dist0=index[0].get_nns_by_vector(vector, moreproduct, search_k=-1, include_distances=True)
    ind1,dist1=index[1].get_nns_by_vector(vector, 1, search_k=-1, include_distances=True)

    # print(ind0,ind1,dist0,dist1)
    #одна только логистическая регрессия иногда ошибается при классификации запроса в сторону товаров,
    # поэтому добавлено условие дистанции до ближайшего вектора - это уточняет результат
    if target==0 and dist0[0]<dist1[0]: 
        if moreproduct>1: #Реализован поиск похожих товаров в контентной части бота
            for i in ind0:
                row=ds[0].loc[i]
                print(f"{row['product_id']} {row['title']} ({info})")
        else: #по умолчанию - поиск одного товара
            row=ds[0].loc[ind0[0]]
            return f"{row['product_id']} {row['title']} ({info})"
    else:
        row=ds[1].loc[ind1[0]]
        answers=[i for i in row['answer'].split('\t') if len(i)>5]
        return f"{answers[np.random.randint(0, high=len(answers))]} ({info if target==1 else 'логистическая регрессия ошиблась при классификации'})"


In [60]:
# поиск наиближайшего товара
get_answer('Юбка ORBY детская')

'58e3cfe6132ca50e053f5f82 Юбка детская ORBY (запрос из товарной группы)'

In [63]:
# поиск нескольких похожих товаров
get_answer('Юбка ORBY детская',moreproduct=5)

58e3cfe6132ca50e053f5f82 Юбка детская ORBY (запрос из товарной группы)
5b07bceb22a44943045a0a62 Юбка детская (запрос из товарной группы)
5ad469cec15ae36576339ca0 Юбка детская (запрос из товарной группы)
5a22a8a3bd36c041b11f0536 Юбка детская (запрос из товарной группы)
5b10d2b3ec98553606317117 Рубашка детская (запрос из товарной группы)


In [64]:
# запрос не из товарной тематики
get_answer('что делать, сегодня нашёл 2 миллиона рублей')

'что не знаешь что делать? тогда отдай тому кто знает, а если серьезно- можешь отнести их в полицию они найдут что с ними сделать, или другой вариант возьми чуть, а остальные надежно спрячь пускай лежат будешь брать сколько необходимо чтобы подозрений не вызывало не у кого от куда у тебя такие деньги.. (запрос не идентифицирован, как товарный, найти ответ из болталки)'

In [65]:
get_answer('парень относиться цветной линза девушка зелёны')

'некоторых очки делают чертовски сексапильными!. (запрос не идентифицирован, как товарный, найти ответ из болталки)'

далее следуют все подготовительные действия, обработка, индексирование, обучение и т.д.

In [2]:
# изучить и обработать продуктовую базу:
pds=pd.read_csv(f'{DIR}ProductsDataset.csv')


In [4]:
pds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35548 entries, 0 to 35547
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           35548 non-null  object 
 1   descrirption    33537 non-null  object 
 2   product_id      35536 non-null  object 
 3   category_id     35536 non-null  float64
 4   subcategory_id  35536 non-null  object 
 5   properties      35536 non-null  object 
 6   image_links     35533 non-null  object 
dtypes: float64(1), object(6)
memory usage: 1.9+ MB


In [6]:
# pds[pds.category_id==22].head(5)
pds.head(5)

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...


Предобработаем ответы mail.ru из файла: выбрать только вопросы и поместить в датасет.
эти вопросы предполагается использовать для обучения логистической регрессии - как тексты, не связанные с товарными запросами(хотя не исключено, что в запросах мейл такие могут присутствовать)

In [21]:
# %%time
# from tqdm.notebook import tqdm
# tqdm.pandas()

from tqdm import tqdm, notebook
import time
df=pd.DataFrame(columns=['question','answer'])

#Мы идем по всем записям, просматриваем содержимое
# и сразу после знака --- находим вопрос
with open(file_name:=f"{DIR}Otvety.txt", "r",buffering=200000000,encoding='utf-8') as fin:
    t = tqdm(total=os.stat(file_name).st_size) # инициализировать прогресбар размером файла
    start_time = time.time() #использовать интервал для настройки частоты отображения прогресбара
    question = False
    written = False
    lastpos=0
    tmpcnt=0
    while line:=fin.readline():
        if time.time()-start_time>2:
            start_time = time.time()
            t.update((fin.tell()-lastpos,lastpos:=fin.tell())[0])
        if tmpcnt>100: break
        if line.startswith("---"):#начало блока вопрос/несколько ответов
            tmpcnt+=1
            written = False
            question = False
            continue
        line=line.replace("\t", " ").replace("<br>", "").strip().lower()
        if question:
            # сохранить ответ в текущий(последний) ряд
            df.iloc[-1]['answer']=df.iloc[-1]['answer']+line +'\t'
            written = True
            continue
        if not question:
            # сохранить вопрос
            question = True
            df=pd.concat([df,pd.DataFrame({'question':[line],'answer':['']})],ignore_index=True)
            continue
    t.update((fin.tell()-lastpos,lastpos:=fin.tell())[0])

display(df)
df.to_csv(f'{DIR}Otvety.csv',index=False)

 96%|█████████▌| 34052/35548 [1:00:16<02:38,  9.42it/s]


,question,answer
0,,
1,вопрос о тдв)) давно и хорошо отдыхаем)) лично...,"хомячка....\tмужика, йопаря, собачку и 50 коше..."
2,как парни относятся к цветным линзам? если у д...,меня вобще прикалывает эта тема :).\tкогда это...
3,"что делать, сегодня нашёл 2 миллиона рублей? .","если это ""счастье "" действительно на вас свали..."
4,эбу в двенашке называется итэлма что за эбу? .,эбу — электронный блок управления двигателем а...
...,...,...
96,посоветуйте спавн меню к сталкер кал оф мизери...,спавн меню уже встроено. делаешь как на картин...
97,а как правильно делать минет? .,галина радионова всему тебя надо учить по два ...
98,как скрыть страницу вкатакти .,мои настройки - приватность.\tникак.\t
99,"санаторий ""петродворец"" (бывшие царские конюшн...",суки поганые себе кормушку устроят.....\tотель...


In [ ]:
df=pd.read_csv(f'{DIR}Otvety.csv')
df.dropna(inplace=True)
print(df.iloc[3]['answer'].split('\t')) #просто посмотреть, как получилось

Теперь нам нужно предобработать текст, чтобы обучить word2vec и получить эмбеддинги. Удаляем знаки препинания и делаем лемматизацию

In [45]:

# эта переменная используется для входных данных Word2Vec
sentences=None

def prep(ds,col):
    # обработака одного столбца датасета функционалом из "preprocess_txt"
    # и формирование дополнительного столбца с обработанным текстом
    def applypreprocess(x):
        # обработка отдельного ряда, отображение прогресбара степени обработки всего датасета
        nonlocal tmpcnt,start_time,lastcnt,sentences,t
        tmpcnt+=1
        if time.time()-start_time>2:
            start_time = time.time()
            #так было удобно сделать в одной строке
            t.update((tmpcnt-lastcnt,lastcnt:=tmpcnt)[0])
        # print(row['title'])
        spls = preprocess_txt(x)
        sentences.append(spls)
        return ' '.join(spls)
    #преобразовать текст столбца col посредством preprocess_txt (исключить пунктуацию, привести к нормальной форме слова) 
    sentences = [] #альтернативный выход функции (дублирует результаты сохранения в дополнительный столбец)
    t = tqdm(total=len(ds.index)) # инициализировать прогресбар 
    start_time = time.time() #использовать интервал для настройки частоты отображения прогресбара
    tmpcnt=0
    lastcnt=0
    ds[f'{col}_prep']=ds.iloc[:][col].apply(applypreprocess)
    return sentences #здесь это не глобальная


In [ ]:
# предобработать столбец title в таблице товаров
sentences = prep(pds,'title')
# сохранить
pds.to_csv(f'{DIR}ProductsDataset_prep.csv')


In [16]:
# Обучим модель word2vec на заголовках товарной таблицы
sentences = [i for i in sentences] # в товарах встречается и одно слово, не пропускать if len(i) > 2
model = Word2Vec(sentences=sentences, vector_size=100, min_count=1, window=5)
model.save("w2v_model_prod")
#загрузить полуобученную модель, (если она сейчас не в памяти)
model = Word2Vec.load("w2v_model_prod")

In [51]:
# предобработать столбец question в таблице -болталке мейлру
sentences = prep(df,'question')


100%|█████████▉| 1163089/1163330 [2:17:58<00:01, 140.50it/s] 


In [54]:
# сохранить
pds.to_csv(f'{DIR}Otvety_prep.csv',index=False)
# sentences можно получить из датасета. но он так же готов и после prep(dxxx,'xxxxx')
sentences = pds.iloc[:]['title_prep'].tolist()
sentences=[i.split() for i in sentences]


In [31]:
#корпус предложений из набора запросов мейлру
sentences = df.iloc[:]['question_prep'].tolist()
sentences=[a for i in sentences if len(a:=i.split())>1] #учитывать запросы более одного слова


[['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести'],
 ['парень',
  'относиться',
  'цветной',
  'линза',
  'девушка',
  'зелёный',
  'глаз',
  'голубой'],
 ['делать', 'найти', '2', 'миллион', 'рубль'],
 ['эбу', 'двенашка', 'называться', 'итэлма', 'эбу'],
 ['академия',
  'вампир',
  'даный',
  'момент',
  'часть',
  'книга',
  'академия',
  'вампир'],
 ['защититься', 'энергетический', 'вампир'],
 ['выращивать',
  'магнолия',
  'открытый',
  'грунт',
  'средний',
  'полоса',
  'россия',
  'вид',
  'сорт',
  'зимовать',
  'укрытие',
  'укрытие',
  'цвести',
  'магнолия',
  'каков',
  'размер',
  'дерево',
  'особенность',
  'выращивание',
  'указывать',
  'ответ',
  'регион',
  'прикреплять',
  'фото'],
 ['отформатировать',
  'диск',
  'c',
  'дос',
  'формат',
  'ntfs',
  'команда',
  'fdisk',
  'жёсткий',
  'диск',
  'найти'],
 ['относиться',
  'парикмахер',
  'работать',
  'дом',
  'помешать',
  'например',
  'присутствие',
  'дом',
  'ребёнок',
  'муж',
  'нормально',
  '

In [79]:
len(sentences) #[:5]

1163330

теперь можно дообучить модель на запросах мейлру

In [83]:

model.build_vocab(corpus_iterable=sentences, update=True)
# продолжить тренировку с новыми данными (корпус запросов мейлру)
model.train(corpus_iterable=sentences, total_examples=len(sentences), epochs=model.epochs)


(67849574, 68636410)

In [63]:
model.wv.most_similar(positive=['юбка','кофта'],negative=['брюки'])

[('рукав', 0.8301815390586853),
 ('юбочка', 0.8221212029457092),
 ('штанишки', 0.8165958523750305),
 ('шорты', 0.8089324831962585),
 ('сарафан', 0.7961568236351013),
 ('кофточка', 0.789705753326416),
 ('маечка', 0.7602289915084839),
 ('обтягивать', 0.7535911798477173),
 ('чулок', 0.7487340569496155),
 ('коротенький', 0.7418197989463806)]

In [60]:
print(model.wv.most_similar.__doc__)

Find the top-N most similar keys.
        Positive keys contribute positively towards the similarity, negative keys negatively.

        This method computes cosine similarity between a simple mean of the projection
        weight vectors of the given keys and the vectors for each key in the model.
        The method corresponds to the `word-analogy` and `distance` scripts in the original
        word2vec implementation.

        Parameters
        ----------
        positive : list of (str or int or ndarray) or list of ((str,float) or (int,float) or (ndarray,float)), optional
            List of keys that contribute positively. If tuple, second element specifies the weight (default `1.0`)
        negative : list of (str or int or ndarray) or list of ((str,float) or (int,float) or (ndarray,float)), optional
            List of keys that contribute negatively. If tuple, second element specifies the weight (default `-1.0`)
        topn : int or None, optional
            Number of top-N 

Один из простейших бейзлайн способов представления фразы/предложения в качестве вектора - использование усредненной суммы векторов слов, полученных кодированием word2vec. 
создадим дополнительные столбцы для обоих датасетов, куда закодируем вектором фразы:
для товаров - обработанный столбец title,
для болталки мейлру - обработанный столбец вопросов.Теперь нам нужно сложить в индекс все вопросы. Используем библиотеку annoy. Проходимся по всем ответам, считаем, что вектор предложения - сумма word2vecов слов, которые входят в него (конечно же усредненная)

In [87]:
index_map = {}
counter = 0
# создать дополнительный столбец с векториндексом запроса (и для товаров, и для НЕ товаров)
def sumword2vec(ds,col):
    # функция обертка для реализации прогресбара
    def applyvec(line):
        # функция для формирования значений вектор-столбца
        nonlocal tmpcnt,start_time,lastcnt,t
        tmpcnt+=1
        if time.time()-start_time>2:
            start_time = time.time()
            #так было удобно сделать в одной строке
            t.update((tmpcnt-lastcnt,lastcnt:=tmpcnt)[0])
        n_w2v = 0
        spls = line.split() #в рядах обработанные строки слов, разделенные пробелом
        
        # вычисление вектора фразы
        vector = np.zeros(100)
        for word in spls:
            if word in model.wv:
                vector += model.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        return vector
    t = tqdm(total=len(ds.index)) # инициализировать прогресбар 
    start_time = time.time() #использовать интервал для настройки частоты отображения прогресбара
    tmpcnt=0
    lastcnt=0
    # создать в датасете дополнительный столбец с вектор-индексом фразы(из title или question)
    ds[f'vecindex']=ds.iloc[:][col].apply(applyvec)
    t.update((tmpcnt-lastcnt,lastcnt:=tmpcnt)[0])
    t.close()



In [97]:
pds.head(1)

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,title_prep,vecindex
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,юбка детский orby,"[1.4841254154841106, 1.0441311125953991, 0.215..."


In [91]:
df.head(1)

,question,answer,question_prep,vecindex
1,вопрос о тдв)) давно и хорошо отдыхаем)) лично...,"хомячка....\tмужика, йопаря, собачку и 50 коше...",вопрос тдв отдыхать лично советовать завести,"[-0.8277268558740616, 0.49365271255373955, 1.2..."


In [90]:
print(df.iloc[0]['vecindex'])


[-0.82772686  0.49365271  1.24050184  1.03491035  1.05339716 -0.18407718
  0.45432561  0.40992025  0.96945837  0.35692266 -0.13263479 -0.01535376
 -1.64355929  0.58240894  0.40453424 -1.08462994 -0.50826946 -0.20484385
 -0.48125024  1.69421851 -0.6142003  -0.36673773 -0.65685503 -0.34903202
 -0.61704534 -1.23975551  1.03734131  0.05972732 -1.06981473 -1.08750941
 -0.69615838 -0.65060829  1.29241284  0.31295301  0.91568514  2.12699525
 -1.3547248  -1.15843924  0.11789038  0.57233183  0.16226711  1.16467658
  0.86137497 -0.95360246  0.18864034 -0.77169943 -0.90862725 -0.3176515
 -0.56394849 -1.13173235  0.29355654  0.16674016 -1.17538136  0.23316197
 -0.6587689  -1.66720207  1.6868925   1.28895317 -0.67346612 -0.18752755
 -0.35471381 -0.67341894  0.58727042 -1.23853339 -0.63254675 -1.23734834
  0.82355444 -0.40675593  0.41626332 -1.9199152   0.66742357 -1.34558213
 -0.55136796 -0.31373637 -0.90782553  0.05098521  0.20856745 -0.11931086
 -0.69548878 -0.37781397 -0.06305252  0.8127331   0.

In [85]:
print(pds.iloc[0]['vecindex'])

[ 0.64734143  0.50432409 -0.23151234 -0.15597232  0.328048   -0.07067225
  0.38690964  0.22729939 -0.91181565  0.11345775 -0.35748898  0.16165511
 -0.06779443 -0.0950877   0.23366391 -0.06760442  0.24694733  0.16366537
 -0.10089072 -0.52164306 -0.24070983  0.05717468  0.24729343 -0.02294074
  0.52298647 -0.28287018 -0.12095548 -0.41508947 -0.24778344 -0.16591442
  0.12029481 -0.40046666  0.35153085 -0.50978494  0.07510399  0.30821139
 -0.07206717 -0.38813299  0.51708973 -0.33726708  0.19357984 -0.49946754
  0.66286455 -0.07805283  0.43857615  0.16608332  0.50298305  0.28398754
 -0.56687314  0.38311675  0.21942645 -0.11375095 -0.18812849  0.12314178
  0.16768277 -0.26240962 -0.10943727 -0.33590893 -0.05984133 -0.30275752
  0.54257871 -0.32819748 -0.35818039 -0.13317965 -0.05576548  0.04662169
 -0.05684746 -0.05069867 -0.39161339 -0.09801223  0.1088431   0.72340854
  0.37337418  0.11233049 -0.17050511 -0.24796922  0.01852107 -0.17917768
 -0.21408011 -0.01815455  0.37436533 -0.04484458 -0

In [96]:
# model.wv['юбченка']
'юбка' in model.wv.key_to_index

True

In [86]:
sumword2vec(pds,'title_prep')



100%|██████████| 35548/35548 [00:01<00:00, 32190.58it/s]


In [89]:
sumword2vec(df,'question_prep')

100%|██████████| 1163330/1163330 [01:44<00:00, 11150.18it/s]


In [92]:
# создание столбца vecindex заняло 2 минуты, а сохранение на диск заняло 24минуты.
# похоже, нет смысла хранить расширенные векторные данные???
pds.to_csv(f'{DIR}ProductsDataset_vec.csv',index=False)
df.to_csv(f'{DIR}Otvety_vec.csv',index=False)
model.save("w2v_model_prod")

по имеющимся в датасетах запросам следует обучить логистическую регрессию, чтобы различать
товарный запрос (из датасета pds столбец vecindex) и
не относящийся к продуктовому запрос (из датасета df вопросов/ответов mailru тоже столбец vecindex)

In [99]:
# соединить датасеты с формированием целевого признака
pds_=pds[['vecindex']].assign(y=lambda x:0) #установить целевую переменную 0 для товаров
df_=df[['vecindex']].assign(y=lambda x:1) #установить целевую переменную 1 для НЕ товаров

In [129]:
# если использовать полные таблицы, то выборка по y=0 и y=1 будет несбалансированная
print(len(pds_.index),len(df_.index))

35548 1163330


In [104]:
df_.head(2)

,vecindex,y
1,"[-0.8277268558740616, 0.49365271255373955, 1.2...",1
2,"[-0.7071204707026482, 0.5916969319805503, 0.97...",1


In [105]:
# взять поровну примеров
pdsdf=pd.concat([pds_,df_.iloc[:len(pds_.index)]])

In [113]:
# расцепить вектор -запроса по разным колонкам
pdsdf[[f'col{i}' for i in range(len(pdsdf.iloc[0]['vecindex']))]]=pd.DataFrame(pdsdf['vecindex'].tolist())
pdsdf.head(1)

C:\Users\vsevolod.volkov\AppData\Local\Temp\ipykernel_6492\1920511805.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pdsdf[[f'col{i}' for i in range(len(pdsdf.iloc[0]['vecindex']))]]=pd.DataFrame(pdsdf['vecindex'].tolist())
C:\Users\vsevolod.volkov\AppData\Local\Temp\ipykernel_6492\1920511805.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pdsdf[[f'col{i}' for i in range(len(pdsdf.iloc[0]['vecindex']))]]=pd.DataFrame(pdsdf['vecindex'].tolist())


,vecindex,y,col0,col1,col2,col3,col4,col5,col6,col7,...,col90,col91,col92,col93,col94,col95,col96,col97,col98,col99
0,"[1.4841254154841106, 1.0441311125953991, 0.215...",0,1.484125,1.044131,0.215191,0.278143,0.962704,0.90284,0.978578,0.071939,...,1.182462,-1.223395,-0.294203,-0.76379,-0.61891,1.142289,-0.446753,0.159297,-0.088929,-0.790881


In [ ]:
clf=LogisticRegression(n_jobs=-1,verbose=1,random_state=0)
X, y = pdsdf.iloc[:, 2:], pdsdf.iloc[:, 1]
X_train, X_valid, y_train, y_valid =  train_test_split(X, y, test_size=0.3, shuffle=True)
clf.fit(X_train,y_train)
y_predict=clf.predict(X_valid)


In [116]:
print(f'f1score={f1_score(y_valid,y_predict)}')

f1score=0.4716630977872948


In [117]:
# mean accuracy очень низкая. с таким же успехом можно гадать 50/50
print(f'score={clf.score(X_train,y_train)}')

score=0.5087708722647537


In [118]:
# регуляризация слабо влияет на результат
clf=LogisticRegression(n_jobs=-1,verbose=1,random_state=0,C=1e5).fit(X_train,y_train)
y_predict=clf.predict(X_valid)
print(f'score={clf.score(X_train,y_train)}')
print(f'f1score={f1_score(y_valid,y_predict)}')


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   11.3s finished


score=0.5087105913557176
f1score=0.4719314938154139


In [128]:
fweight=len(pds_.index)/len(df_.index)
fweight

0.030557107613488864

In [126]:
fweight=len(pds_.index)/len(df_.index)
# создать колонку весов для испольбзования в дальнейшем несбалансированного датасета
pdsdffull=pd.concat([pds_,df_]).assign(weight=lambda dataset: (dataset.y==0).map({True:1,False:fweight}))
pdsdffull.sample(2)

,vecindex,y,weight
5380,"[0.5139991500681, -0.018002582048731192, 0.512...",1,0.030557
377102,"[0.17393629591573367, 0.8586087786507878, 0.72...",1,0.030557


In [131]:
# расцепить вектор -запроса по разным колонкам
pdsdffull[[f'col{i}' for i in range(len(pdsdffull.iloc[0]['vecindex']))]]=pd.DataFrame(pdsdffull['vecindex'].tolist())
pdsdffull.head(1)

C:\Users\vsevolod.volkov\AppData\Local\Temp\ipykernel_6492\3898240133.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pdsdffull[[f'col{i}' for i in range(len(pdsdffull.iloc[0]['vecindex']))]]=pd.DataFrame(pdsdffull['vecindex'].tolist())
C:\Users\vsevolod.volkov\AppData\Local\Temp\ipykernel_6492\3898240133.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pdsdffull[[f'col{i}' for i in range(len(pdsdffull.iloc[0]['vecindex']))]]=pd.DataFrame(pdsdffull['vecindex'].tolist())
C:\Users\vsevolod.volkov\AppData\Local\Temp\ipykerne

,vecindex,y,weight,col0,col1,col2,col3,col4,col5,col6,...,col90,col91,col92,col93,col94,col95,col96,col97,col98,col99
0,"[1.4841254154841106, 1.0441311125953991, 0.215...",0,1.0,1.484125,1.044131,0.215191,0.278143,0.962704,0.90284,0.978578,...,1.182462,-1.223395,-0.294203,-0.76379,-0.61891,1.142289,-0.446753,0.159297,-0.088929,-0.790881


In [209]:
clf=LogisticRegression(n_jobs=-1,verbose=1,random_state=0,C=1e5)
X, y = pdsdffull.iloc[:, 2:], pdsdffull.iloc[:, 1]
# сюда попал столбец с весами weight, который не предполагается использовать как фичи
X_train, X_valid, y_train, y_valid =  train_test_split(X, y, test_size=0.3, shuffle=True)
# первый столбец использовать для весов и не использозвать для фичей
clf.fit(X_train.iloc[:, 1:].to_numpy(),y_train.to_numpy(),sample_weight=X_train.iloc[:, 0])
y_predict=clf.predict(X_valid.iloc[:, 1:].to_numpy())
print(f'score={clf.score(X_train.iloc[:, 1:].to_numpy(),y_train.to_numpy(),sample_weight=X_train.iloc[:, 0])}')
print(f'f1score={f1_score(y_valid,y_predict)}')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.8min finished


score=0.9305743689438893
f1score=0.9505958429838532


score=0.9079662636705298
f1score=0.9507234025953372
такой результат значительно лучше, уже можно использовать для различения категории запроса: товарный или НЕ товарный,
причем всего-лишь по одному вектору, являющемуся средним для фразы всех слов в запросе

In [170]:

print(choosetarget('юбка и брюки')[2])
print(choosetarget('где бы попить поесть')[2])

запрос из товарной группы
запрос не идентифицирован, как товарный, найти ответ из болталки


In [149]:
# проиндексировать двумя разными индексаторами оба датасета по столбцам vecindex и сохранить


def prepindex(ds,annoyindex):
    def applyvec(row):
        # построить annoy индекс для векторов, характеризующих запросы (отдельно товарные и НЕ товарные)
        nonlocal tmpcnt,start_time,lastcnt,t,annoyindex
        tmpcnt+=1
        if time.time()-start_time>2:
            start_time = time.time()
            #так было удобно сделать в одной строке
            t.update((tmpcnt-lastcnt,lastcnt:=tmpcnt)[0])
        annoyindex.add_item(row.name, row.vecindex)

    t = tqdm(total=len(ds.index)) # инициализировать прогресбар 
    start_time = time.time() #использовать интервал для настройки частоты отображения прогресбара
    tmpcnt=0
    lastcnt=0
    # сопоставить сохранить индекс каждого ряда с вектор-индексом фразы(из title или question)
    _=ds.iloc[:].apply(applyvec,axis=1)
    t.update((tmpcnt-lastcnt,lastcnt:=tmpcnt)[0])
    t.close()


In [150]:
indexProduct = annoy.AnnoyIndex(100 ,'angular')
prepindex(pds,indexProduct)
indexProduct.build(10) 


100%|██████████| 35548/35548 [00:03<00:00, 11481.21it/s]


In [192]:
indexFlood = annoy.AnnoyIndex(100 ,'angular')
prepindex(df,indexFlood)
indexFlood.build(30,n_jobs=-1)


100%|██████████| 1163330/1163330 [01:20<00:00, 14409.55it/s]


True

Теперь остается реализовать метод, который получит на вход вопрос и найдет ответ к нему!
Мы препроцессим вопрос, находим ближайший вопрос и выбираем ответ на ближайший вопрос.

In [58]:
get_answer('Юбка ORBY детская')

'58e3cfe6132ca50e053f5f82 Юбка детская ORBY (запрос из товарной группы)'

In [57]:
get_answer('Где ключи от танка')

'key &amp; crack (alawar) видео помощь. (логистическая регрессия ошиблась при классификации)'

In [56]:
get_answer('куплю детские шапки')

'57de544096ad842e26de8027 Продам детские шапки (запрос из товарной группы)'

In [31]:
print(index.get_nns_by_vector.__doc__)

Returns the `n` closest items to vector `vector`.

:param search_k: the query will inspect up to `search_k` nodes.
`search_k` gives you a run-time tradeoff between better accuracy and speed.
`search_k` defaults to `n_trees * n` if not provided.

:param include_distances: If `True`, this function will return a
2 element tuple of lists. The first list contains the `n` closest items.
The second list contains the corresponding distances.


In [53]:
df.loc[2].answer

'меня вобще прикалывает эта тема :).\tкогда этобыло редкость - было забавно, а когда все знают, что эта фальшивка, то уже не прикольно, как силиконовые сиськи или как налепленные синтетические волосы.\t'

In [20]:
display(df.head(5))

,question_prep,answer
1,вопрос тдв отдыхать лично советовать завести,"хомячка....\tмужика, йопаря, собачку и 50 коше..."
2,парень относиться цветной линза девушка зелёны...,меня вобще прикалывает эта тема :).\tкогда это...
3,делать найти 2 миллион рубль,"если это ""счастье "" действительно на вас свали..."
4,эбу двенашка называться итэлма эбу,эбу — электронный блок управления двигателем а...
5,академия вампир даный момент часть книга акаде...,"4. охотники и жертвы, ледяной укус, поцелуй ть..."


In [287]:
display(pds.sample(2))

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,title_prep,vecindex
14169,Футболка новая L,"Новая футболочка, ткань приятная к телу, не мн...",594b89dc2138bbd8f34f0732,9.0,909,{'futbolki_topy_tip': 'Футболки'},http://cache3.youla.io/files/images/360_360/59...,футболка новый l,"[0.3155921896298726, -0.4812539617220561, 0.97..."
31432,Платье Турция 44 размер,Классное качество,5a9af387f6957692d20bdeb2,9.0,904,{},http://cache3.youla.io/files/images/360_360/5a...,платье турция 44 размер,"[0.6597742699086666, -0.9200384765863419, 1.57..."


In [288]:
# сохранить
model.save(f'{DIR}word2vec.model')
word_vectors = model.wv
word_vectors.save(f'{DIR}word2vec.wordvectors')

indexFlood.save(f'{DIR}indexFlood.annoy')
indexProduct.save(f'{DIR}indexProduct.annoy')

df[['question_prep','answer']].to_csv(f'{DIR}mailruflood.csv')
pds[['title','product_id','title_prep']].to_csv(f'{DIR}ProductsDS.csv')

In [291]:
pickle.dump(clf, open(f'{DIR}LogisticRegression.sav', 'wb'))